<a href="https://colab.research.google.com/github/santiagoahl/Digit-recognition/blob/main/k_means.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prev

In [ ]:
def create_means(k, dim, mean_bounds):
  min = mean_bounds[0]
  max = mean_bounds[1]
  means = np.random.uniform(min, max, size=(k, dim))
  return means

In [ ]:
 np.random.uniform(0, 1, size=(10, 64))

In [ ]:
def assignate(data, means):
  assignations = []
  for point in data:
    distances = [np.linalg.norm(point-means[i]) for i in range(len(means))]
    point_cluster = np.argmin(distances)
    assignations.append(point_cluster)
  return assignations

In [ ]:
def clusterize(data, assignations, k):
  clusters = [[] for _ in range(k)]
  for j in range(len(assignations)):
    assignation = assignations[j]
    point = data[j]
    clusters[assignation].append(point)
  return clusters

In [ ]:
def update_means(data, clusters, old_means, k, dim, mean_bounds):
  means = [np.zeros(2) for _ in range(k)]
  i=0
  for cluster in clusters:
    if len(cluster)==0:
      min = mean_bounds[0]
      max = mean_bounds[1]
      means[i] = np.random.uniform(min, max, size=(k, dim)) #old_means[i]
      #print(f'{i} is empty {old_means[i]}')
      i+=1
      continue
    mean = np.mean(cluster, axis=0)
    means[i] = mean
    i+=1
    #print(np.array(cluster).shape)
  return np.array(means, dtype=object)

In [ ]:
def delta(i, j):
  if i==j:
    return 0.0
  else:
    return 1.0

In [ ]:
def loss_function(data, assignations, clusters, means, k):
  loss = 0
  for i in range(len(means)):
    mean = means[i]
    for j in range(len(data)):
      point = data[j]
      #print(f'j:{j}, las:{len(assignations)}, pm:{len(point)},{len(mean)}')
      loss += delta(i, assignations[j])*np.linalg.norm(point-mean)
  return loss/len(data)

In [ ]:
def fit_k_means(data, k, dim, mean_bounds,max_iter=100, tolerance=1e-2, live_plot=False, metrics=False):
  means = create_means(k=k, dim=dim, mean_bounds=mean_bounds)
  for _ in range(max_iter):
    assignations = assignate(data, means)
    clusters = clusterize(data, assignations, k)
    old_means = means
    means = update_means(data, clusters, means, k, dim, mean_bounds)
    #print(means)
    change = np.mean([np.linalg.norm(means[i]-old_means[i]) for i in range(len(means))])
    if change < tolerance:
      break
    if metrics==True:
      print(f'Change: {change}')
      print(f'Loss: {loss_function(data, assignations, clusters, means, k)}')
    if live_plot==True:
      summary = pd.DataFrame({
        'x': data[:, 0],
        'y': data[:, 1],
        'cluster': assignations
      })
      plt.figure(figsize=(6, 6))
      sns.scatterplot(data=summary, x='x', y='y',hue='cluster')
      plt.scatter(means[:,0], means[:,1], color='r')
      plt.show()
  return np.array(assignations)

# OOP

In [ ]:
print('v1.5.8')

In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [ ]:
class KMeans():
  def __init__(self, k):
    self.k = k

  def fit(self, data, mean_bounds=(0, 1.0), max_iter=100000, tolerance=1e-2, live_plot=False, verbose=0):
    self.data = data
    self.dim = len(self.data[0])
    self.normalize()
    means = self.create_means(mean_bounds=mean_bounds)
    for _ in range(max_iter):
      self.assignations = self.assignate(means)
      clusters = self.clusterize(self.assignations)
      old_means = means
      print(means)
      means = self.update_means(clusters, mean_bounds)
      #print(means)
      change = np.mean([np.linalg.norm(means[i]-old_means[i]) for i in range(len(means))])
      if change < tolerance:
        break
      if verbose==1:
        print(f'Change: {change} | Loss: {self.loss(self.assignations, clusters, means)} \n \n')
      if live_plot==True:
        summary = pd.DataFrame({
          'x': self.data[:, 0],
          'y': self.data[:, 1],
          'cluster': self.assignations
        })
        plt.figure(figsize=(6, 6))
        sns.scatterplot(data=summary, x='x', y='y',hue='cluster')
        plt.scatter(means[:,0], means[:,1], color='r')
        plt.show()
    return np.array(self.assignations)
    
  def loss(self, assignations, clusters, means):
    loss = 0
    for i in range(len(means)):
      mean = means[i]
      for j in range(len(self.data)):
        point = self.data[j]
        #print(f'j:{j}, las:{len(assignations)}, pm:{len(point)},{len(mean)}')
        loss += self.delta(i, assignations[j])*np.linalg.norm(point-mean)
    return loss/len(self.data)

  def delta(self, i, j):
    if i==j:
      return 0.0
    else:
      return 1.0
      
  def update_means(self, clusters, mean_bounds):
    means = [np.zeros(self.dim) for _ in range(self.k)]
    i=0
    for cluster in clusters:
      if len(cluster)==0:
        min = mean_bounds[0]
        max = mean_bounds[1]
        means[i] = np.random.uniform(min, max, size=(self.k, self.dim)) #old_means[i]
        #print(f'{i} is empty {old_means[i]}')
        i+=1
        continue
      mean = np.mean(cluster, axis=0)
      means[i] = mean
      i+=1
      #print(np.array(cluster).shape)
    return np.array(means, dtype=object)

  def clusterize(self, assignations):
    clusters = [[] for _ in range(self.k)]
    for j in range(len(assignations)):
      assignation = assignations[j]
      point = self.data[j]
      clusters[assignation].append(point)
    return clusters

  def assignate(self, means):
    assignations = []
    for point in self.data:
      distances = [np.linalg.norm(point-means[i]) for i in range(len(means))]
      point_cluster = np.argmin(distances)
      assignations.append(point_cluster)
    return assignations

  def create_means(self, mean_bounds):
    min = mean_bounds[0]
    max = mean_bounds[1]
    means = np.random.uniform(min, max, size=(self.k, self.dim))
    return means
          
  def normalize(self):
    sc = MinMaxScaler()
    sc.fit(self.data)
    self.data = sc.transform(self.data)

  def sort_assignations(self, old, targets):
    data = self.assignations
    new = data
    for x in range(len(data)):
      #print('\n')
      observation = data[x]
      #print(f'observation:{observation}')
      for i in range(len(old)):
        #print(i)
        #print(old[i], observation)
        if old[i] == observation:
          #print(f'OK! old:{old[i]}, ob:{observation}')
          new[x] = targets[i]
    self.assignations = new

In [ ]:
def f(x):
  x=x+2